In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

In [9]:
with open('../data/HangJeongDong_ver20230101.geojson.txt', 'r', encoding='utf-8') as f:
  file = f.readlines()
  f.close

In [10]:
concat_file = ''.join(file)
type(concat_file)

str

In [11]:
map_dict = eval(concat_file)
map_dict.keys()

dict_keys(['type', 'name', 'crs', 'features'])

In [12]:
map_dict['features'][0]['properties'].keys()

dict_keys(['OBJECTID', 'adm_nm', 'adm_cd', 'adm_cd2', 'sgg', 'sido', 'sidonm', 'temp', 'sggnm', 'adm_cd8'])

In [13]:
sidonms = list()
for dong in map_dict['features']:
  sidonms.append(dong['properties']['sidonm'])
print(set(sidonms))

{'인천광역시', '강원도', '제주특별자치도', '경상북도', '충청남도', '세종특별자치시', '경상남도', '대구광역시', '충청북도', '서울특별시', '대전광역시', '전라남도', '전라북도', '경기도', '울산광역시', '부산광역시', '광주광역시'}


In [14]:
new_features = list()
for dong in map_dict['features']:
  if dong['properties']['sidonm'] == '대전광역시' or dong['properties']['sidonm'] == '충청남도' or dong['properties']['sidonm'] == '세종특별자치시':
    new_features.append(dong)
map_dict['features'] = new_features

In [15]:
def polygon_area(lats, lons, algorithm = 0, radius = 6378137):
    """
    Computes area of spherical polygon, assuming spherical Earth. 
    Returns result in ratio of the sphere's area if the radius is specified.
    Otherwise, in the units of provided radius.
    lats and lons are in degrees.
    """
    import numpy as np
    from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff, deg2rad
    lats = np.deg2rad(lats)
    lons = np.deg2rad(lons)
    # Line integral based on Green's Theorem, assumes spherical Earth
    #close polygon
    if lats[0]!=lats[-1]:
        lats = append(lats, lats[0])
        lons = append(lons, lons[0])
    #colatitudes relative to (0,0)
    a = sin(lats/2)**2 + cos(lats)* sin(lons/2)**2
    colat = 2*arctan2( sqrt(a), sqrt(1-a) )
    #azimuths relative to (0,0)
    az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)
    # Calculate diffs
    # daz = diff(az) % (2*pi)
    daz = diff(az)
    daz = (daz + pi) % (2 * pi) - pi
    deltas=diff(colat)/2
    colat=colat[0:-1]+deltas
    # Perform integral
    integrands = (1-cos(colat)) * daz
    # Integrate 
    area = abs(sum(integrands))/(4*pi)
    area = min(area,1-area)
    if radius is not None: #return in units of radius
        return area * 4*pi*radius**2
    else: #return in ratio of sphere total area
        return area

In [16]:
poligon = np.array([(-106.936763,35.958191),(-106.944385,35.239293),(-106.452396,35.281908),(-106.407844,35.948708)])
polygon_area(poligon[:,0], poligon[:,1])/1000000

1257.4734536618341

In [18]:
size_list = list()
for dong in map_dict['features']:
  sidonm = dong['properties']['sidonm']
  temp = dong['properties']['temp']
  dong_poligon = np.array(dong['geometry']['coordinates'][0][0])
  size = polygon_area(dong_poligon[:,1], dong_poligon[:,0])/1000000
  
  size_list.append([sidonm, temp, size])
size_df = pd.DataFrame(size_list, columns = ['시도', '구/군 읍면동', '면적(km^2)'])
size_df

,시도,구/군 읍면동,면적(km^2)
0,대전광역시,동구 효동,2.087250
1,대전광역시,동구 판암1동,6.044410
2,대전광역시,동구 판암2동,0.761826
3,대전광역시,동구 용운동,3.504477
4,대전광역시,동구 자양동,1.161701
...,...,...,...
307,세종특별자치시,세종시 연기면,12.247587
308,세종특별자치시,세종시 해밀동,8.475248
309,세종특별자치시,세종시 연동면,21.741566
310,세종특별자치시,세종시 반곡동,18.377297


In [21]:
size_df['시/군/구'] = size_df['구/군 읍면동'].apply(lambda x: x.split(' ')[0])
size_df['읍면동'] = size_df['구/군 읍면동'].apply(lambda x: x.split(' ')[1])

In [24]:
size_df['읍면동'].loc[11] = '중앙동 (대전)'
size_df['읍면동'].loc[11]

'중앙동 (대전)'

In [52]:
size_df = size_df[['읍면동', '면적']]
print(size_df[size_df['읍면동'] == '중앙동'])
size_df[size_df['읍면동'] == '중앙동 (대전)']

    읍면동        면적
94  중앙동  0.778324


,읍면동,면적
11,중앙동 (대전),1.305748


In [25]:
size_df.to_csv('대전충남세종_행정동별 면적.csv')